In [1]:
import yaml
from pytorch_lightning.loggers import NeptuneLogger
from Modules.train import DataModule, Model, get_trainer


c:\Users\197796\Anaconda3\envs\AISeed\lib\site-packages\neptune\common\warnings.py:62: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
  warnings.warn(


# Training

In [2]:
# %%script false --no-raise-error

task = "detection"

neptune_logger = NeptuneLogger(
        project="kaori/AISeed",
        # api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIyZjZiMDA2YS02MDM3LTQxZjQtOTE4YS1jODZkMTJjNGJlMDYifQ==",
        log_model_checkpoints=False,
        tags=[task]
    )

with open("Modules/config.yaml", 'r') as stream:
    PARAMS = yaml.safe_load(stream)
    PARAMS = PARAMS['detect']
    print(PARAMS)



{'architect_settings': {'name': 'model-test', 'backbone': {'name': 'maskrcnn-m', 'is_full': False, 'is_pretrained': True, 'is_freeze': False}, 'n_cls': 2}, 'dataset_settings': {'name': 'PennFudan', 'path': 'data/PennFudanPed', 'img_size': 320}, 'training_settings': {'gpu_ids': [0], 'n_gpu': 1, 'loss': 'none', 'ckpt_path': './mode/test', 'n_epoch': 50, 'n_batch': 2, 'num_workers': 12, 'optimizer': 'adam', 'lr_scheduler': 'step', 'early_stopping': False, 'lr': 0.0001, 'lr_step': 10, 'lr_decay': 0.8, 'momentum': 0.9, 'weight_decay': 0.005}}


In [2]:
import os
for key in os.environ:
    print(key)

ALLUSERSPROFILE
APPDATA
APPLICATION_INSIGHTS_NO_DIAGNOSTIC_CHANNEL
CHROME_CRASHPAD_PIPE_NAME
COMMONPROGRAMFILES
COMMONPROGRAMFILES(X86)
COMMONPROGRAMW6432
COMPUTERNAME
COMSPEC
CONDA_DEFAULT_ENV
CONDA_EXE
CONDA_EXES
CONDA_PREFIX
CONDA_PROMPT_MODIFIER
CONDA_PYTHON_EXE
CONDA_ROOT
CONDA_SHLVL
DRIVERDATA
EFC_19008
ELECTRON_RUN_AS_NODE
HOMEDRIVE
HOMEPATH
JAVA_HOME
JPY_INTERRUPT_EVENT
LOCALAPPDATA
LOGONSERVER
NUMBER_OF_PROCESSORS
ONEDRIVE
ONEDRIVECOMMERCIAL
ORIGINAL_XDG_CURRENT_DESKTOP
OS
PATH
PATHEXT
PROCESSOR_ARCHITECTURE
PROCESSOR_IDENTIFIER
PROCESSOR_LEVEL
PROCESSOR_REVISION
PROGRAMDATA
PROGRAMFILES
PROGRAMFILES(X86)
PROGRAMW6432
PROMPT
PSMODULEPATH
PUBLIC
PYDEVD_IPYTHON_COMPATIBLE_DEBUGGING
PYTHONIOENCODING
PYTHONUNBUFFERED
PYTHONUTF8
SESSIONNAME
SYSTEMDRIVE
SYSTEMROOT
TEMP
TMP
USERDNSDOMAIN
USERDOMAIN
USERDOMAIN_ROAMINGPROFILE
USERNAME
USERPROFILE
VSCODE_AMD_ENTRYPOINT
VSCODE_CODE_CACHE_PATH
VSCODE_CRASH_REPORTER_PROCESS_TYPE
VSCODE_CWD
VSCODE_HANDLES_UNCAUGHT_ERRORS
VSCODE_IPC_HOOK
VSC

In [3]:
# %%script false --no-raise-error

import albumentations as A
from albumentations.pytorch import ToTensorV2

img_size = PARAMS['dataset_settings']['img_size']
transforms_img = A.Compose([
    A.RandomResizedCrop(img_size, img_size),  # Random crop and resize
    A.HorizontalFlip(),  # Random horizontal flip
    A.VerticalFlip(),  # Random vertical flip
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),  # Convert to tensor
])

data = DataModule(PARAMS['dataset_settings'], PARAMS['training_settings'], [None, None])

In [4]:
# %%script false --no-raise-error
neptune_logger.log_hyperparams(params=PARAMS)
model = Model(PARAMS=PARAMS, task = task)
trainer = get_trainer(PARAMS['training_settings'], neptune_logger)
# 
trainer.fit(model, data)

c:\Users\197796\Anaconda3\envs\AISeed\lib\site-packages\neptune\common\warnings.py:62: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  warnings.warn(


NeptuneMissingApiTokenException: 

----NeptuneMissingApiTokenException-------------------------------------------

The Neptune client couldn't find your API token.

You can get it here:
    - https://app.neptune.ai/get_my_api_token

There are two options to add it:
    - specify it in your code
    - set an environment variable in your operating system.

CODE
Pass the token to the init_run() function via the api_token argument:
    neptune.init_run(project='WORKSPACE_NAME/PROJECT_NAME', api_token='YOUR_API_TOKEN')

ENVIRONMENT VARIABLE (Recommended option)
or export or set an environment variable depending on your operating system:

    Linux/Unix
    In your terminal run:
        export NEPTUNE_API_TOKEN="YOUR_API_TOKEN"

    Windows
    In your CMD run:
        set NEPTUNE_API_TOKEN="YOUR_API_TOKEN"

and skip the api_token argument of the init_run() function:
    neptune.init_run(project='WORKSPACE_NAME/PROJECT_NAME')

You may also want to check the following docs pages:
    - https:

In [1]:
import os
os.environ.get("NEPTUNE_API_TOKEN")

# Testing

In [ ]:
%%script false --no-raise-error
import torch
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks
import os
from torchvision import transforms

with open("models/class_name.txt", "r", encoding='utf-8') as f:
    class_names = f.read().splitlines()

model_list = [name.split('.')[0] for name in os.listdir("models") if name.endswith('ckpt')]
model_list += ["fcn", "resnet", "fasterrcnn", "maskrcnn"]
inv_normalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.255],
    std=[1/0.229, 1/0.224, 1/0.255]
)

def predict(image, model_choice):
    labels, detection = None, None
    with open("Modules/config.yaml", 'r') as stream:
            PARAMS = yaml.safe_load(stream)
            PARAMS = PARAMS['segment']
    PARAMS['architect_settings']['backbone']['is_full'] = True
    if(model_choice == "fcn"):
        PARAMS['architect_settings']['backbone']['name'] = 'fcn-m'    
        model = Model(PARAMS, task="segmentation")
    elif(model_choice == "resnet"):
        PARAMS['architect_settings']['backbone']['name'] = 'resnet-s'    
        model = Model(PARAMS, task="classification")
    elif(model_choice == "fasterrcnn"):
        PARAMS['architect_settings']['backbone']['name'] = 'fasterrcnn-s'    
        model = Model(PARAMS, task="detection")
    elif(model_choice == "maskrcnn"):
        PARAMS['architect_settings']['backbone']['name'] = 'maskrcnn-s'    
        model = Model(PARAMS, task="detection")
    else:
        model = Model.load_from_checkpoint(f"models/{model_choice}.ckpt").cpu()
    model.eval()
    transforms = model.model.preprocess
    tensor_image = transforms(image)
    with torch.no_grad():
        y_hat = model(tensor_image.unsqueeze(0))
        if(model.task == "classification"):
            preds = torch.softmax(y_hat, dim=-1).tolist()
            labels = {class_names[k]: float(v) for k, v in enumerate(preds[0][:-1])}
        elif(model.task == "segmentation"):
            num_classes = y_hat.shape[1]
            masks = y_hat[0]
            classes_masks = masks.argmax(0) == torch.arange(num_classes)[:, None, None]
            tensor_image = inv_normalize(tensor_image)
            detection = draw_segmentation_masks((tensor_image * 255.).to(torch.uint8), 
                                              masks=classes_masks, alpha=.6)
            detection = detection.numpy().transpose(1, 2, 0) / 255.
        elif(model.task == "detection"):
            if("maskrcnn" in model_choice):
                boolean_masks = [out['masks'][out['scores'] > .75] > 0.5
                                for out in y_hat][0]
                detection = draw_segmentation_masks((tensor_image * 255.).to(torch.uint8),
                                                    boolean_masks.squeeze(1), alpha=0.8)
                detection = detection.numpy().transpose(1, 2, 0) / 255.
            else:
                detection = draw_bounding_boxes((tensor_image * 255.).to(torch.uint8), 
                                                    boxes=y_hat[0]["boxes"][:5],
                                                    colors="red",
                                                    width=5)
                detection = detection.numpy().transpose(1, 2, 0) / 255.

    return labels, detection

In [ ]:
%%script false --no-raise-error

import gradio as gr

title = "Application Demo "
description = "# A Demo of Wrapping Pretrained Networks"
example_list = [["examples/" + example] for example in os.listdir("examples")]


with gr.Blocks() as demo:
    demo.title = title
    gr.Markdown(description)
    with gr.Row():
        with gr.Column():
            model = gr.Dropdown(model_list, label="Select Model", interactive=True)
            im = gr.Image(type="pil", label="input image")
            label_conv = gr.Label(label="Predictions", num_top_classes=4)
        with gr.Column():
            im_detection = gr.Image(type="pil", label="Detection")
            btn = gr.Button(value="predict")
    btn.click(predict, inputs=[im, model], outputs=[label_conv, im_detection])
    gr.Examples(examples=example_list, inputs=[im, model], outputs=[label_conv, im_detection])
      

demo.launch(share=True)